In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
df=pd.read_csv('../input/titanic/train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
df['Age'].fillna(df['Age'].median(),inplace=True)

In [7]:
df['Embarked'] = df['Embarked'].fillna(df.Embarked.mode().iloc[0])


In [8]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [9]:
train=df.drop(['Name','Cabin','Ticket'],axis=1)
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [10]:
y=train.Survived
train_df=train.drop('Survived',axis=1)

train_df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,male,22.0,1,0,7.2500,S
1,2,1,female,38.0,1,0,71.2833,C
2,3,3,female,26.0,0,0,7.9250,S
3,4,1,female,35.0,1,0,53.1000,S
4,5,3,male,35.0,0,0,8.0500,S


In [11]:
s=(train_df.dtypes=='object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['Sex', 'Embarked']


In [12]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()
enc.fit(train_df[['Sex','Embarked']])
train_df[['Sex','Embarked']] = enc.transform(train_df[['Sex','Embarked']])
train_df.head(10)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,1.0,22.0,1,0,7.2500,2.0
1,2,1,0.0,38.0,1,0,71.2833,0.0
2,3,3,0.0,26.0,0,0,7.9250,2.0
3,4,1,0.0,35.0,1,0,53.1000,2.0
4,5,3,1.0,35.0,0,0,8.0500,2.0
5,6,3,1.0,28.0,0,0,8.4583,1.0
6,7,1,1.0,54.0,0,0,51.8625,2.0
7,8,3,1.0,2.0,3,1,21.0750,2.0
8,9,3,0.0,27.0,0,2,11.1333,2.0
9,10,2,0.0,14.0,1,0,30.0708,0.0


In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train_df,y,train_size=0.8,test_size=0.2, random_state=0)

In [14]:
from sklearn.metrics import mean_absolute_error

In [15]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=1000, random_state=0)
clf.fit(X_train, y_train)
predict0=clf.predict(X_test)
mae0=mean_absolute_error(predict0,y_test)
print('mean absolute error =',mae0)

mean absolute error = 0.1452513966480447


In [16]:
from sklearn import svm
clf1 = svm.SVC()
clf1.fit(X_train, y_train)
predict1=clf1.predict(X_test)
mae1=mean_absolute_error(predict1,y_test)
print('mean absolute error =',mae1)


mean absolute error = 0.3407821229050279


In [17]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
predict2=neigh.predict(X_test)
mae2=mean_absolute_error(predict2,y_test)
print('mean absolute error =',mae2)


mean absolute error = 0.31843575418994413


In [18]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
predict3=gnb.predict(X_test)
mae3=mean_absolute_error(predict3,y_test)
print('mean absolute error =',mae3)


mean absolute error = 0.2122905027932961


In [19]:
from sklearn.tree import DecisionTreeClassifier
clf4 = DecisionTreeClassifier(random_state=0,max_leaf_nodes=100,max_depth=100)
clf4.fit(X_train, y_train)
predict4=gnb.predict(X_test)
mae4=mean_absolute_error(predict4,y_test)
print('mean absolute error =',mae4)

mean absolute error = 0.2122905027932961


In [20]:
test=pd.read_csv('../input/titanic/test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [21]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [22]:
test['Age'].fillna(test['Age'].median(),inplace=True)

In [23]:
test['Fare'].fillna(test['Fare'].median(),inplace=True)

In [24]:
test_df=test.drop(['Name','Cabin','Ticket'],axis=1)
test_df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S


In [25]:
test_df.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [26]:
s=(test_df.dtypes=='object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['Sex', 'Embarked']


In [27]:
enc = OrdinalEncoder()
enc.fit(test_df[['Sex','Embarked']])
test_df[['Sex','Embarked']] = enc.transform(test_df[['Sex','Embarked']])
test_df.head(10)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1.0,34.5,0,0,7.8292,1.0
1,893,3,0.0,47.0,1,0,7.0000,2.0
2,894,2,1.0,62.0,0,0,9.6875,1.0
3,895,3,1.0,27.0,0,0,8.6625,2.0
4,896,3,0.0,22.0,1,1,12.2875,2.0
5,897,3,1.0,14.0,0,0,9.2250,2.0
6,898,3,0.0,30.0,0,0,7.6292,1.0
7,899,2,1.0,26.0,1,1,29.0000,2.0
8,900,3,0.0,18.0,0,0,7.2292,0.0
9,901,3,1.0,21.0,2,0,24.1500,2.0


In [28]:
test_df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1.0,34.5,0,0,7.8292,1.0
1,893,3,0.0,47.0,1,0,7.0000,2.0
2,894,2,1.0,62.0,0,0,9.6875,1.0
3,895,3,1.0,27.0,0,0,8.6625,2.0
4,896,3,0.0,22.0,1,1,12.2875,2.0
...,...,...,...,...,...,...,...,...
413,1305,3,1.0,27.0,0,0,8.0500,2.0
414,1306,1,0.0,39.0,0,0,108.9000,0.0
415,1307,3,1.0,38.5,0,0,7.2500,2.0
416,1308,3,1.0,27.0,0,0,8.0500,2.0


In [29]:
#model_1.fit(X, y)
predictions = clf.predict(test_df)

output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
output.to_csv('RandomForest_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
